In [1]:
import pandas as pd
import itertools

### Count Label

In [2]:
data = pd.read_excel('data/occupation_exposure_eval.xlsx')
data.head()

,Occupation,Responden 1,Responden 2,Responden 3,Responden 4,Responden 5,Responden 6
0,Legislators,1,1,2,1,2,2
1,Finance Managers,1,2,2,3,3,3
2,Agricultural and Forestry Production Managers,1,2,1,1,3,2
3,Hotel Managers,1,2,2,1,3,3
4,Physicists and Astronomers,2,3,3,1,3,3


### Fleiss Kappa All Keywords

In [3]:
k = 3 # number of category
N = len(data) # number of data
respondents = ["Responden 1", "Responden 2", "Responden 3", "Responden 4", "Responden 5", "Responden 6"]

def count_categories(row):
    tidak_terdampak = row[row == 1].count()
    terdampak_sebagian = row[row == 2].count()
    terdampak_penuh = row[row == 3].count()
    return pd.Series({'tidak terdampak': tidak_terdampak,
                      'terdampak sebagian': terdampak_sebagian,
                      'terdampak penuh': terdampak_penuh})

def calculate_k(dummy_data, n):
    # Calculate pi for each row
    list_pi = []
    for index, row in dummy_data.iterrows():
        pi = 1 / (n*(n-1)) * (row['tidak terdampak']**2 + row['terdampak sebagian']**2 + row['terdampak penuh']**2 - n)
        list_pi.append(pi)

    # Calculate pj
    pj_tidak_terdampak = 1 / (N*n) * dummy_data['tidak terdampak'].sum()
    pj_terdampak_sebagian = 1 / (N*n) * dummy_data['terdampak sebagian'].sum()
    pj_terdampak_penuh = 1 / (N*n) * dummy_data['terdampak penuh'].sum()

    # Calculate p and pe
    p = 1 / N * sum(list_pi)
    pe = pj_tidak_terdampak**2 + pj_terdampak_sebagian**2 + pj_terdampak_penuh**2

    # Calculate k
    k = (p-pe) / (1-pe)
    return k

def lookfor_k(dummy_data):
    for r in range(1, len(respondents) + 1):
        combinations = itertools.combinations(respondents, r)
        print(f"Combinations of {r} respondent(s):")
        for combo in combinations:
            selected_columns = [col for col in data.columns if col in combo]
            dummy_data = data[selected_columns].copy()
            n = len(dummy_data.columns)
            if n == 1:
                continue  # Skip calculation and printing if only one column
            dummy_data = dummy_data.apply(count_categories, axis=1)
            k = calculate_k(dummy_data, n)
            print("Nilai k untuk kombinasi"+ str(combo) + " = " + str(k))

lookfor_k(data)

Combinations of 1 respondent(s):
Combinations of 2 respondent(s):
Nilai k untuk kombinasi('Responden 1', 'Responden 2') = 0.09947643979057588
Nilai k untuk kombinasi('Responden 1', 'Responden 3') = 0.3336561743341404
Nilai k untuk kombinasi('Responden 1', 'Responden 4') = 0.298913043478261
Nilai k untuk kombinasi('Responden 1', 'Responden 5') = 0.00967296176877016
Nilai k untuk kombinasi('Responden 1', 'Responden 6') = -0.3708053691275169
Nilai k untuk kombinasi('Responden 2', 'Responden 3') = 0.22952218430034144
Nilai k untuk kombinasi('Responden 2', 'Responden 4') = -0.08517350157728705
Nilai k untuk kombinasi('Responden 2', 'Responden 5') = 0.32142857142857145
Nilai k untuk kombinasi('Responden 2', 'Responden 6') = 0.08388814913448735
Nilai k untuk kombinasi('Responden 3', 'Responden 4') = 0.00967296176877016
Nilai k untuk kombinasi('Responden 3', 'Responden 5') = 0.16469594594594592
Nilai k untuk kombinasi('Responden 3', 'Responden 6') = 0.0004151100041512018
Nilai k untuk kombinas

In [4]:
N = len(data) # number of data
n = 5 # number of respondent (change this)
k = 3 # number of category
dummy_data = data[["Responden 1", "Responden 2", "Responden 3", "Responden 4", "Responden 5"]]
dummy_data = dummy_data.apply(count_categories, axis=1)

# Calculate pi for each row
list_pi = []
for index, row in dummy_data.iterrows():
    pi = 1 / (n*(n-1)) * (row['tidak terdampak']**2 + row['terdampak sebagian']**2 + row['terdampak penuh']**2 - n)
    list_pi.append(pi)

# Calculate pj
pj_tidak_terdampak = 1 / (N*n) * dummy_data['tidak terdampak'].sum()
pj_terdampak_sebagian = 1 / (N*n) * dummy_data['terdampak sebagian'].sum()
pj_terdampak_penuh = 1 / (N*n) * dummy_data['terdampak penuh'].sum()

# Calculate p and pe
p = 1 / N * sum(list_pi)
pe = pj_tidak_terdampak**2 + pj_terdampak_sebagian**2 + pj_terdampak_penuh**2
print("p = " + str(p))
print("pe = " + str(pe))

# Calculate k
k = (p-pe) / (1-pe)
print("k = " + str(k))

p = 0.4767441860465118
pe = 0.39586803677663607
k = 0.13387166081787605


### Cohen's Kappa

In [5]:
dummy_data = data[["Responden 1", "Responden 2", "Responden 3", "Responden 4", "Responden 5"]]
# Dictionary mapping old column names to new column names
# new_column_names = {
#     'Responden 5': 'Responden 4',
#     'Responden 6': 'Responden 5'
# }
# # Rename columns using rename() method
# dummy_data = dummy_data.rename(columns=new_column_names)

def observed_agreement(i, j):
    confusion_matrix = pd.crosstab(dummy_data['Responden '+str(i)], dummy_data['Responden '+str(j)])
    confusion_matrix = confusion_matrix.reindex(index=[1, 2, 3], columns=[1, 2, 3], fill_value=0)
    diagonal_sum = sum(confusion_matrix.iloc[i,i] for i in range(len(confusion_matrix)))
    return diagonal_sum / len(dummy_data)

def chance_agreement(i, j):
    df = dummy_data[['Responden '+str(i), 'Responden '+str(j)]]
    sum_rows = [sum(row) for row in df.values]
    sum_columns = [sum(col) for col in df.values.transpose()]
    p_i = sum([row * row for row in sum_rows]) / (sum(sum_rows) ** 2)
    p_j = sum([col * col for col in sum_columns]) / (sum(sum_columns) ** 2)
    return p_i * p_j

def count_kappa(i, j):
    # Calculate observed agreement
    observed_agr = observed_agreement(i, j)
    # Calculate chance agreement
    chance_agr = chance_agreement(i, j)
    # Calculate Cohen's Kappa
    kappa = (observed_agr - chance_agr) / (1 - chance_agr)
    return kappa

def average(numbers):
    total = sum(numbers)
    return total / len(numbers)

list = []    
for i in range(1, 5):  # Looping pertama dari 1 sampai 4
    for j in range(i+1, 6):  # Looping kedua dari nilai i sampai 5
        k = count_kappa(i, j)
        list.append(k)
        print(f'nilai kappa responden {i} dengan responden {j} = {k}')

print(average(list))

nilai kappa responden 1 dengan responden 2 = 0.4815276558178813
nilai kappa responden 1 dengan responden 3 = 0.6228331926659222
nilai kappa responden 1 dengan responden 4 = 0.6466411180956707
nilai kappa responden 1 dengan responden 5 = 0.41073595007420793
nilai kappa responden 2 dengan responden 3 = 0.5051117094641301
nilai kappa responden 2 dengan responden 4 = 0.3402809650203752
nilai kappa responden 2 dengan responden 5 = 0.5522011772505215
nilai kappa responden 3 dengan responden 4 = 0.41089273320503134
nilai kappa responden 3 dengan responden 5 = 0.4579051522293501
nilai kappa responden 4 dengan responden 5 = 0.2695121054702033
0.4697641759293293
